# kvanc Websocket-Server

Nächste Schritte
- Doc-String dort wo es einen braucht einbauen, Code muss strukturiert sein. (klar und verständlich)
- Clean code
- ~~Protkoll überlegen —> Wir wollen 1:1, an wenn muss man es schicken: simon.marcin:meldung: —> Meldung auseinander nehmen können und dann schauen wie man es schicken kann. UDP-Übung~~
- ~~Websockets sind anonym —> Websocket-Objekt anschauen, Source IP herausfinden. Init clients_connected muss ein dict sein, kein set mehr. Get ip out of python object. Was haben wir alles in der connection drin?~~
- ~~Pattern mit 1:1 & 1:* (broadcast), Gruppenname. join.gruppename, es reicht 1:1 und 1:* für Kommunikation~~
- ~~done: Session wird beendet , User-name beendet, Server beenden muss noch gemacht werden~~~ 
- ~~User name muss drin sei bei der ersten Message die man schickt. Abspeichern von Username —> ins dict speichern. (siehe TCP übung~~~
- ~~done: Websocket objekt aus dem dict löschen wenn Connection beende~~~
- ~~Username im dict auf Name auflösen~~~

In [1]:
import asyncio
import os
import websockets
import nest_asyncio
nest_asyncio.apply() # rausnehmen
from re import search

class WebsocketServer:
    """
    This is a websocket client which can only receives messages from a websocket server.
    """

    def __init__(self, server_ip="$USER", server_port=8080):
        """
        Init of Websocket_Client
        server_ip str: Either and IPv4 address or a name which can get resolved to an IP address.
        server_port int: default port is 80
        user_name str: Your user_name within the chat --> user_name="$USER"
        """
        #self.server_ip = server_ip
        self.server_port = server_port
        self.clients_connected = dict() # key:name, value:websocketobject
        self.server_ip = os.path.expandvars(server_ip)
        self.all_user_names = None
        self.get_all_user_names()
        self.websocket_server = websockets.serve(self.receiver,
                                                 self.server_ip, self.server_port)
        print("Init of WebsocketServer.")
    
    def get_all_user_names(self):
        with open('/etc/hosts') as f:
             read_data = f.read()                
        self.all_user_names = read_data
        #print(self.all_user_names[:300])
    
    ################ Create 1:1-Communication ################
    #async def receiveMessage(self, message):
    #    print(f"< {message}")

    #async def sendMessage(self):
    #    message = f"Hello Client, your message is received!!"
    #    await asyncio.sleep(2) # sleep for 2 seconds before returning message
    #    return message
    #
    #async def receiveMessage_handler(self, websocket):
    #    async for msg in websocket:
    #        await self.receiveMessage(msg)
    #        print('1. message from client received')

    #async def sendMessage_handler(self, websocket):
    #    while True:
    #        message = await self.sendMessage()
    #        await websocket.send(message)
    #        print('2. message sent to client')
    #        #await asyncio.sleep(2)
    #        
    ## sending/receiving combined in one handler
    #async def message_handler(self, websocket):
    #    receive_task = asyncio.ensure_future(
    #        self.receiveMessage_handler(websocket))
    #    send_task = asyncio.ensure_future(
    #        self.sendMessage_handler(websocket))
    #    done, pending = await asyncio.wait(
    #        [receive_task, send_task],
    #        return_when=asyncio.FIRST_COMPLETED,
    #    )
    #    for task in pending:
    #        task.cancel()
            
    ################ Create Broadcast-Communication ################
    async def broadcast(self):
        await asyncio.sleep(1)
        while True:
            await asyncio.gather(
                *[websocket.send("Hello, this is a broadcast communication. Welcome!") for websocket in self.clients_connected],
                return_exceptions=False
            )
            await asyncio.sleep(2)
        
    # using broadcasting
    async def connection_handler(self, websocket):
        user_name = await websocket.recv()
        print(f"addr: {websocket.remote_address}")
        self.clients_connected[user_name] = websocket # value : websocketobject
        print(f"{websocket} added to the chat.\n")
        
        try:
            async for msg in websocket:
                print(f"Message from client: {msg}") # username: julia.mueller
                sender, receiver, message = msg.split(':', maxsplit=2)
                print(f"split into sender:{sender}, receiver:{receiver}, message:{message}")
                
                self.clients_connected[sender] = self.clients_connected.pop(user_name) # safe user in dict with name
                print(f"All Clients connected: {self.clients_connected}\n")
                    
                if receiver == "*": #broadcast send to all clients
                    print(f"A message will be sent to all receipients.\n")
                    for user in self.clients_connected: 
                        if user != self.server_ip:
                            await websocket.send(f"Reply from server to all: {message}") # an alle
                else:
                    if receiver in self.clients_connected: # send to one client
                        await websocket.send(f"Reply from server to {receiver} : {message}") # an receiver
                    else:
                        await websocket.send(f"{receiver} does not exist. Try sending it to another receiver.")
                
        except websockets.ConnectionClosedError:
            pass
        
        finally:
            print(f"Client {websocket} disconnected.")
            del self.clients_connected[user_name]
        
    ################ Receiver of Handler ################
    async def receiver(self, websocket, path):
        await self.connection_handler(websocket)
        #await self.message_handler(websocket)

        
################ Start Websocket-Server ################
server = WebsocketServer(server_ip='julia.mueller')
loop = asyncio.get_event_loop()

asyncio.create_task(server.broadcast())
loop.run_until_complete(server.websocket_server)
# Simons Code
#asyncio.ensure_future(server.websocket_server)

Init of WebsocketServer.


addr: ('10.35.148.180', 48128)
<websockets.server.WebSocketServerProtocol object at 0x7f172ef82790> added to the chat.

